In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.regularizers import l2
import cv2
import matplotlib.pyplot as plt

IMG_SIZE = 224
classes = ['Melanoma', 'NotMelanoma']

base_model = MobileNetV3Large(
    include_top=False,
    weights=None,
    include_preprocessing=True
)

model_testing = Sequential([
    Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
    base_model,
    GlobalAveragePooling2D(),
    Dense(1024, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.5),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    Dropout(0.3),
    Dense(1, activation='sigmoid')
])

model_testing.load_weights('model_default_aug.h5')

def dull_razor_hair_removal(img):
    img_np = np.array(img)

    img_gray = cv2.cvtColor(img_np, cv2.COLOR_RGB2GRAY)

    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    blackhat = cv2.morphologyEx(img_gray, cv2.MORPH_BLACKHAT, kernel)

    lowbound = 12
    _, mask = cv2.threshold(blackhat, lowbound, 255, cv2.THRESH_BINARY)

    inpaint_radius = 3
    img_final = cv2.inpaint(img_np, mask, inpaint_radius, cv2.INPAINT_TELEA)
    
    return img_final

def predict_single_image(model, image_path, classes, true_label, remove_hair=False):
    img = load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))

    if remove_hair:
        processed_img = dull_razor_hair_removal(img)
        img_for_pred = processed_img.astype('float32')
    else:
        processed_img = img_to_array(img)
        img_for_pred = processed_img

    img_array = np.expand_dims(img_for_pred, axis=0)

    prediction = model.predict(img_array)[0][0]
    if prediction < 0.5:
        pred_class = classes[0]
    else:
        pred_class = classes[1]

    plt.figure(figsize=(5, 5))
    if remove_hair:
        plt.imshow(processed_img.astype('uint8'))
    else:
        plt.imshow(img)
    plt.title(f"True label: {true_label} | Predicted label: {pred_class}")
    plt.axis("off")
    plt.show()
    
    return pred_class

image_path = "data test/notmelanoma_test.jpg"
true_label = classes[1]  # 0 untuk Melanoma, 1 untuk NotMelanoma

predicted_class = predict_single_image(model_testing, image_path, classes, true_label, remove_hair=True)